In [ ]:
# pip freeze > requorements.txt

# Introduction
ARIMA (AutoRegressive Integrated Moving Average) model is a popular time series analysis and forecasting method used in econometrics, finance, and other fields. It is a class of statistical models that captures the stochastic nature of a time series and its underlying patterns.

The ARIMA model has three components:

Autoregression (AR) - A regression model that uses lagged values of the time series to predict its current value.

Moving Average (MA) - A model that uses the past errors of the time series to predict its current value.

Integration (I) - A differencing process that transforms a non-stationary time series into a stationary one by computing the differences between consecutive observations.

The notation for an ARIMA model is ARIMA(p, d, q), where:

* p: the order of the autoregressive part
* d: the degree of differencing
* q: the order of the moving average part

The selection of the values for p, d, and q is typically done using statistical methods such as the Akaike information criterion (AIC) or the Bayesian information criterion (BIC). Once the optimal values for p, d, and q are determined, the ARIMA model can be used to make predictions about future values of the time series. 

### Assumptions

>Stationarity: the time series should be stationary. This means that the mean, variance, and autocorrelation of the series should remain constant over time. If the series is non-stationary, it can be made stationary through differencing.

>No seasonality: ARIMA models assume that the time series does not have any seasonal patterns. If there is seasonality in the data, a seasonal ARIMA (SARIMA) model should be used instead.

>No outliers: ARIMA models assume that there are no extreme values or outliers in the time series. Outliers can be identified and removed from the data to improve the accuracy of the model.

>No autocorrelation in residuals: The residuals (the difference between the observed values and the predicted values) should be independent and identically distributed (i.e., have no autocorrelation). If there is autocorrelation in the residuals, it indicates that the model is not capturing all the information in the data.

>Normally distributed residuals: The residuals should be normally distributed with a mean of zero. If the residuals are not normally distributed, it may indicate that the model is not appropriate for the data or that there are other factors affecting the series that are not accounted for in the model.

### Steps

* Import the necessary libraries: import pandas, numpy, matplotlib, and statsmodels libraries.

* Load the data: Load the data into a pandas DataFrame and set the 'Date' column as the index.

* Convert the data to time series: Convert the DataFrame into a time series using the 'to_datetime' function.

* Visualize the data: Plot the data using the 'plot' function to visualize the trend, seasonality, and any other patterns in the data.

* Stationarize the data: Check if the time series is stationary using the 'adf_test' function from the statsmodels library. If the time series is not stationary, you can apply differencing to make it stationary.

* Determine the parameters: Determine the values of p, d, and q for the ARIMA model by analyzing the ACF and PACF plots.

* Build the ARIMA model: Build the ARIMA model using the 'ARIMA' function from the statsmodels library, passing in the values of p, d, and q.

* Fit the model: Fit the model to the data using the 'fit' method.

* Forecast: Forecast the values for April 2024 using the 'forecast' method, passing in the number of periods you want to forecast.

In [2]:
# import relevant libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

import warnings
warnings.filterwarnings("ignore")

# 2. Data

In [9]:
# load data and parse data column
euro = pd.read_csv('EURO.csv', parse_dates=['Date'], index_col='Date')

# historical data for exogenous variables
df_inflation = pd.read_csv('/home/munyao/Desktop/flat_iron_school/clients/lusa/data/Inflation Rates.csv')
df_inflation['Date'] = pd.to_datetime(df_inflation['Year'], format='%Y')
df_inflation.set_index('Date', inplace=True)

# foreign trade
file_path = "/home/munyao/Desktop/flat_iron_school/clients/lusa/data/Foreign Trade Summary (Ksh Million).csv"
df_foreign_trade = pd.read_csv(file_path)
df_foreign_trade['Date'] = pd.to_datetime(df_foreign_trade['Year'], format='%Y')
df_foreign_trade.set_index('Date', inplace=True) 

#  GDP
file_path = "/home/munyao/Desktop/flat_iron_school/clients/lusa/data/Annual GDP.csv"
gdp_df = pd.read_csv(file_path)
gdp_df = gdp_df.iloc[:-1]  
gdp_df['Date'] = pd.to_datetime(gdp_df['Year'], format='%Y')
gdp_df.set_index('Date', inplace=True) 

### 2.0.1 Data Profile Report

In [4]:
# import neccesary libraries
from pandas_profiling import ProfileReport

# generate data report using the pandas profiling
euro_profile = ProfileReport(euro, title="EURO Data Data Report")
foreign_trade_profile = ProfileReport(df_foreign_trade, title="Foreign Trade Data Data Report")
gdp_profile = ProfileReport(gdp_df, title="EURO Data Data Report")

### 2.0.2 Datasets Mergeing

,Month,Annual Average Inflation,12-Month Inflation
Year,,,
2023,January,7.95,8.98
2022,December,7.66,9.06
2022,November,7.38,9.48
2022,October,7.48,9.59
2022,September,6.81,9.18
...,...,...,...
2005,May,14.61,14.78
2005,April,13.76,16.02
2005,March,13.07,14.15


In [ ]:
# merge the datasets
df = pd.merge(df_exchange, df_inflation, how='left', left_index=True, right_index=True)
#df = pd.merge(df, df_unemployment, how='left', left_index=True, right_index=True)


## 2.1 Data Cleaning

In [ ]:
# preview EURO data frame
euro.shape

##### Check for Missing values and Duplicates

In [ ]:
# duplicated rows 
print(f"The data frame has {euro.duplicated().sum()} duplicated rows")
print()
# drop the duplicated rows based on all columns
df = euro.drop_duplicates()

print(f"The data frame has {df.duplicated().sum()} duplicated rows after dropping the duplicates")


##### Check for Outliers

In [ ]:
# import neccesary libraries
import seaborn as sns

# create a boxplot for each column
plt.figure(figsize=(10, 8))
sns.boxplot(data=df[['Mean', 'Buy', 'Sell']], palette='Set3')

# set the plot title and y-axis label
plt.title('Boxplot of Exchange Rates')
plt.ylabel('Exchange Rate')

# show the plot
plt.show()


>No outliers detected

## 3.  Data Visualization
visualize the trend, seasonality, and any other patterns in the data.

### 3.1.Seasonality

In [ ]:
# def stationery_display(df,cols):
#     # set figure size
#     plt.figure(figsize=(8, 8))

#     # plot the rolling mean and rolling standard deviation
#     buy = df['Buy']
#     rolmean = buy.rolling(window=12).mean()
#     rolstd = buy.rolling(window=12).std()
#     plt.plot(buy, color='blue', label='Buy')
#     plt.plot(rolmean, color='red', label='Rolling Mean')
#     plt.plot(rolstd, color='black', label='Rolling Std')
#     plt.title('Rolling Mean & Standard Deviation of Buy')
#     plt.xlabel('Date')
#     plt.ylabel('Exchange Rate')
#     plt.legend()

#     # show plot
#     plt.show()
        
# stationery_display(euro,"Mean")        

#### 3.2. Autocorrelation
The ACF plot shows the correlation of the time series with its lagged values. The PACF plot shows the correlation of the time series with its lagged values after removing the effects of the intermediate lags.

Identify the values for p and q: 
* The value of p is the lag at which the PACF plot first crosses the upper confidence interval. 
* The value of q is the lag at which the ACF plot first crosses the upper confidence interval.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# increase plot size
plt.rcParams["figure.figsize"] = (10, 5)

# plot ACF
plot_acf(df['Mean'], lags=30)

plt.axhline(y=-1.96/np.sqrt(len(df)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(df)), linestyle='--', color='gray')
plt.title('Autocorrelation Function (ACF) of Mean')
plt.xlabel('Lags')
plt.ylabel('Correlation')
plt.show()

# Plot PACF
plot_pacf(df['Mean'], lags=30)
plt.axhline(y=-1.96/np.sqrt(len(df)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(df)), linestyle='--', color='gray')
plt.title('Partial Autocorrelation Function (PACF) of Mean')
plt.xlabel('Lags')
plt.ylabel('Correlation')
plt.show()


>The ACF plot shows a slow decay, and the PACF plot shows a sharp cutoff after lag p, a first-order difference is needed to make the time series stationary. 

#### 3.3. Stationarity.

####  Stationarity Test

In [ ]:
# function to test 
def test_stationarity(series):
    # augmented Dickey-Fuller test
    adf_result = adfuller(series)
    print('Augmented Dickey-Fuller Test:')
    print('ADF Statistic: %f' % adf_result[0])
    print('p-value: %f' % adf_result[1])
    print('Critical Values:')
    for key, value in adf_result[4].items():
        print('\t%s: %.3f' % (key, value))
    if adf_result[0] < adf_result[4]['5%']:
        print('ADF test indicates series is stationary')
    else:
        print('ADF test indicates series is non-stationary')

    # kwiatkowski-Phillips-Schmidt-Shin test
    kpss_result = sm.tsa.stattools.kpss(series)
    print('\nKwiatkowski-Phillips-Schmidt-Shin Test:')
    print('KPSS Statistic: %f' % kpss_result[0])
    print('p-value: %f' % kpss_result[1])
    print('Critical Values:')
    for key, value in kpss_result[3].items():
        print('\t%s: %.3f' % (key, value))
    if kpss_result[0] < kpss_result[3]['5%']:
        print('KPSS test indicates series is stationary')
    else:
        print('KPSS test indicates series is non-stationary')

# call function
test_stationarity(df['Mean'])        

## 4. Data PreProcessing

In [ ]:
# drop currency column
final_df = euro.drop("Currency", axis=1)

#####  Apply Differencing
The time series is non-stationary, apply differencing to make it stationary.I will apply first-order differencing.

In [ ]:
# apply first order differencing
diff_y = final_df.diff(periods=1).dropna()

>**After applying differencing, check for stationarity again, and these are results of the Augmented Dickey-Fuller test, the Kwiatkowski-Phillips-Schmidt-Shin test, and the Phillips-Perron test.**

In [ ]:
# call function to test stationarity
test_stationarity(diff_y["Mean"])


## 5. Determine the Parameters

In [ ]:
# import neccesary library
from pmdarima.arima import auto_arima

# Find the optimal order of the ARIMA model
model = auto_arima(diff_y["Mean"], seasonal=False, trace=True,
                   suppress_warnings=True, error_action="ignore")
model

## 6. Modelling
### kesur_model

In [ ]:
# build the ARIMA model
# AR order
p = None  

# differencing order
d = None  

# MA order
q = None  

kesur_mean = sm.tsa.ARIMA(df['Mean'], order=(2, 0, 3))
kesur_sell = sm.tsa.ARIMA(df['Sell'], order=(2, 0, 3))
kesur_buy = sm.tsa.ARIMA(df['Buy'], order=(1, 0, 3))

# fit the model
mean_results = kesur_mean.fit()
sell_results = kesur_sell.fit()
buy_results = kesur_buy.fit()


### Forcast for Feb 24 2024

In [ ]:
euro.head(5)

In [ ]:
# forecast for 1 year
forecast_periods = 12
mean_forecast = mean_results.forecast(steps=forecast_periods)
sell_forecast = sell_results.forecast(steps=forecast_periods)
buy_forecast = buy_results.forecast(steps=forecast_periods)

# print the forecasted values
if len(mean_forecast) > 0:
    print("Mean forecast for April 2024:", mean_forecast.mean())
else:
    print("Error: Mean forecast array is empty")
    
if len(sell_forecast) > 0:
    print("Sell forecast for April 2024:", sell_forecast.mean())
else:
    print("Error: Sell forecast array is empty")

if len(buy_forecast) > 0:
    print("Buy forecast for April 2024:", buy_forecast.mean())
else:
    print("Error: Buy forecast array is empty")


### Tune for Optimised Model

# 7. Evaluation

In [ ]:
# import neccesary libraries
from scipy import stats

# evaluate the model
residuals = mean_results.resid

# plot histogram of residuals
fig, ax = plt.subplots(figsize=(8, 4))
ax.hist(residuals, bins=25, density=True, alpha=0.6, color='b')

# plot normal distribution curve
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, np.mean(residuals), np.std(residuals))
ax.plot(x, p, 'k', linewidth=2)
ax.set_xlabel('Residuals')
ax.set_ylabel('Density')
ax.set_title('Histogram of Residuals')
plt.show()


In [ ]:
# evaluate the model
residuals = sell_results.resid

# plot histogram of residuals
fig, ax = plt.subplots(figsize=(8, 4))
ax.hist(residuals, bins=25, density=True, alpha=0.6, color='b')

# plot normal distribution curve
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, np.mean(residuals), np.std(residuals))
ax.plot(x, p, 'k', linewidth=2)
ax.set_xlabel('Residuals')
ax.set_ylabel('Density')
ax.set_title('Histogram of Residuals')
plt.show()


In [ ]:
# evaluate the model
residuals = buy_results.resid

# plot histogram of residuals
fig, ax = plt.subplots(figsize=(8, 4))
ax.hist(residuals, bins=25, density=True, alpha=0.6, color='b')

# plot normal distribution curve
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, np.mean(residuals), np.std(residuals))
ax.plot(x, p, 'k', linewidth=2)
ax.set_xlabel('Residuals')
ax.set_ylabel('Density')
ax.set_title('Histogram of Residuals')
plt.show()


>residuals are normally distributed